In [30]:
import os
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from gensim.models import Word2Vec

In [13]:
base_dir = os.getcwd()
target_dir = os.path.join(base_dir, 'essay')

print(target_dir)

c:\learn\star\StarLight\MachineLearning\recommendation\simple-art\essay


In [27]:
essays = []

for file in os.listdir(target_dir):
    with open(os.path.join(target_dir, file), 'r', encoding='utf-8') as f:
        content = f.read()
        essays.append(content)

In [22]:
essays

['时之彼处岸之未央\n\n\u3000\u3000如果你听得见我所说的。\n\n\u3000\u3000在遇见你之前，我一直以为爱是一件与运气有关的事。我把心事记录在书页间，焚烧往天国，等待去世的故人替我转交给天使。可是投递去天堂的书信，大概是被不小心分配至哪个无人查看的信箱，我静待多年，却从来没有人回复我的愿望。\n\n\u3000\u3000也许带着愿望去天堂的天使，能成功抵达的只有幸运的几位吧？那么，运气不好的我所写下的纸条，一定被莫名地遗忘在了人世间的某个地方。\n\n\u3000\u3000因为这样，我才见不到你吧。\n\n\u3000\u3000即便你终于来到我身边，可我已经不相信那个世界的存在了。那个需要运气才可以看见的世界。一定没有为我而留的位置吧？\n\n\u3000\u3000你不要趁着我沉睡时，与妖精们窃语。我并未抵达梦之彼处，只是想借着睡梦中不经意的口吻，告诉你。我内心的期望其实没有画上休止符。\n\n\u3000\u3000我怀有不甘的梦，是因为我明明在等待你的到来，却又害怕比邻之幸，将在离别之时蔓延成苦楚。假若真是如此，不如让不幸的我，一直不幸。\n\n\u3000\u3000我偷偷告诉你，我常常会害怕。我会害怕气球。\n\n\u3000\u3000他们轻如幸福，倘若手心有轻微的颤抖，他们便在你笨拙的捕捉间逃走。\n\n\u3000\u3000我害怕你给我的幻梦，也如手中那束气球。纵然绚烂如烟火，也会因为我胆怯的手心，使之于顷刻。\n\n\u3000\u3000我亦害怕，我日渐膨胀的愿望经不起细小的真相去触碰。\n\n\u3000\u3000你该怎样劝说这样的我。逃跑至天涯不再理睬这样任性懦弱胆小充满忧愁战战兢兢期待着你却又害怕着你不能给我永远的我。还是守护着等待着用你温暖的手掌一点一点填补我内心的空缺一步一步教会我小心翼翼滴驶向幸福的来路？\n\n\u3000\u3000后来我无数次想起那一天，你将盛宴呈现在我眼前，光之氤氲在小兔子们的瞳孔里，博物之间萤火如星星般浮在眼前。\n\n\u3000\u3000我丢失在岁月里的信件，在你问着我“你相信我么”的时候，终于抵达到我所希冀的地点。\n\n\u3000\u3000如果你听得见我说。请你不要责怪我。\n\n\u3000\u3000我仍然懦弱笨拙胆小一无是处没法变成让你不用担心的女孩。你

In [32]:
def get_mean_vector(words, model):
    word_vectors = [model.wv[w] for w in words if w in model.wv]
    if len(word_vectors) == 0:
        return None
    return sum(word_vectors) / len(word_vectors)

In [38]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(essays)

# 主题聚类
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(tfidf_matrix)
topics = kmeans.labels_

# Word2Vec训练
sentences = [re.findall(r'\w+', essay.lower()) for essay in essays]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

# 标签系统
tags = ['友谊', '家庭', '爱情', '成长', '励志', '人生', '理想', '青春', '读书', '旅行']

# 关键词匹配推荐
def keyword_recommendation(keyword):
    matching_indices = []
    for i, essay in enumerate(essays):
        if keyword in essay:
            matching_indices.append(i)
    return [essays[i] for i in matching_indices]

# Word2Vec相似度推荐
def w2v_recommendation(ref_essay, topn=10):
    ref_vector = get_mean_vector(re.findall(r'\w+', ref_essay.lower()), w2v_model)
    if ref_vector is None:
        return []
    similarities = [cosine_similarity([ref_vector], [w2v_model.wv[w]])[0][0] for w in w2v_model.wv.index_to_key]
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:topn]
    return [(essays[indices], similarities[indices]) for indices in top_indices]

# 主题/标签筛选推荐
def topic_tag_recommendation(topic_id=None, tags=None):
    filtered = [(i, essay) for i, essay in enumerate(essays) if (topic_id is None or topics[i] == topic_id) and
                (tags is None or all(tag in essay for tag in tags))]
    return [essay for i, essay in filtered]

# 示例用法
keyword = '友谊'
recommended = keyword_recommendation(keyword)
print(f"与关键词'{keyword}'相关的作文:")
for essay in recommended[:3]:
    print(essay[:100], '...\n')

ref_essay = essays[20]
recommended = w2v_recommendation(ref_essay)
print(f"参考作文是:\n{ref_essay[:100]}...")
print(f"与参考作文相似的作文:")
for essay, sim in recommended[:3]:
    print(f"{essay[:100]}... 相似度:{sim:.2f}\n")

print("主题5并包含'友谊'和'家庭'标签的作文:")    
recommended = topic_tag_recommendation(topic_id=5, tags=['友谊', '家庭'])
for essay in recommended[:3]:
    print(essay[:100], '...\n')

c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


与关键词'友谊'相关的作文:
Title: 落叶

[wave]“落叶[/wave][wave]满地念念碎，落叶有存在的意义；秋雨点点恼人愁，秋雨有存在的意义；满月凄凄惹人醉，满月有存在的意义。可是每个人的存在又有着什么意义，每个 ...

参考作文是:
Title: 纸的幻想

“上床睡觉吧。”妈妈催着我上床睡觉。“好。”一躺在床上，我便进入了梦乡。突然，我不知怎么就醒来，咦，我这是到了哪儿呀？怎么躺在一棵大树下？前边是一个[wave]热闹非凡[/w...
与参考作文相似的作文:
时之彼处岸之未央

　　如果你听得见我所说的。

　　在遇见你之前，我一直以为爱是一件与运气有关的事。我把心事记录在书页间，焚烧往天国，等待去世的故人替我转交给天使。可是投递去天堂的书信，大概是被不小... 相似度:1.00

信任自己

　　小时侯，我和妈妈经常玩一种叫“信任后仰”的游戏，规则很简单：直挺挺地猝然后仰，相信在脑浆开花之前，对方一定会伸出双手等待着你。

　　小时侯，我还不懂“信任”是什么，却知道在幼儿园门口... 相似度:1.00

Title: 秋菊

早晨，当你踏入校园的第一步时，你会感觉到，一阵清香扑鼻而来，紧接着，一片[wave]红花衬绿树[/wave]的景象[wave]映入眼[/wave][wave]帘[/wave]。这... 相似度:1.00

主题5并包含'友谊'和'家庭'标签的作文:
